In [ ]:
! pip install kaggle

Load Kaggle api

In [ ]:
!mkdir -p ~/ .kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': Not a directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:04<00:00, 26.2MB/s]
100% 80.9M/80.9M [00:04<00:00, 18.7MB/s]


In [ ]:
# extract compressed dataset which we got in above step
from zipfile import ZipFile
dataset = "sentiment140.zip"
with ZipFile(dataset, 'r') as z:
  z.extractall()
  print('Done')

Done


In [ ]:
# import libararies required
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split # for spliting data to be used for our model training and testing
from sklearn.linear_model import LogisticRegression # our model
from sklearn.metrics import accuracy_score

In [ ]:
# get stopwards like I, we , you etc so that we will remove these from our dataset bcuz they will not make any diff in our prediction
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Data processing
# load data into pandas dataframes
twiter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
twiter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# twiter_data.info()
# twiter_data.shape
twiter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# from the above step we saw that our column names were treated as data row so we need to pass names explicly
c_names = ["Sentiment", "id", "Date", "Flag", "Username", "Tweet"]
twiter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=c_names)
twiter_data.head()

,Sentiment,id,Date,Flag,Username,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# now check if we have any missing values in our data as if any then we have to either drop that row or fill that cell with dummy data
twiter_data.isnull().sum()
# we found nothing :)


,0
Sentiment,0
id,0
Date,0
Flag,0
Username,0
Tweet,0


In [ ]:
# now let's check how many tweets are negative and how many are positive
twiter_data['Sentiment'].value_counts()
# we got 8 lakh postive and 8 lakh negative tweets 0 for -ve and 4 for +ve

,count
Sentiment,
0,800000
4,800000


In [ ]:
# let's replace 4 with 1 just for our easiness
twiter_data['Sentiment'].replace({4: 1}, inplace=True)
twiter_data['Sentiment'].value_counts()

<ipython-input-12-fe6d681b503d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  twiter_data['Sentiment'].replace({4: 1}, inplace=True)


,count
Sentiment,
0,800000
1,800000


In [ ]:
# now we will do stemming on our dataset
# stimming means that we will basically reduce word to thier root
# for example running will be converted to run, basically we will remove everything like numbers, special chars, verbs etc and will keep only keywords
port_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))



In [ ]:
def stemming(content):
  words = re.findall(r'[a-zA-Z]+', content.lower())
  return ' '.join(port_stem.stem(word) for word in words if word not in stop_words)


# update the tweets column of dataset with new tweet stemmed data
twiter_data['Stem_Tweet'] = twiter_data['Tweet'].map(stemming)

In [ ]:
twiter_data.head()

,Sentiment,id,Date,Flag,Username,Tweet,Stem_Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
# from the above table you can see that we need only two cols 1. Sentiment, 2. Stem_Tweet
X = twiter_data['Stem_Tweet']
Y = twiter_data['Sentiment']

In [ ]:
# now let's split our data which we need for testing and training our model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
Y_test

,Sentiment
131348,0
1142114,1
244564,0
445353,0
415893,0
...,...
178459,0
1515130,1
1449952,1
441063,0


In [ ]:
# now our ML model will not understand textual data we need to convert data to numbers so that our model can understand
# for this purpose we are using TfidfVectorizor
vector = TfidfVectorizer()
X_train = vector.fit_transform(X_train)
X_test = vector.transform(X_test)


In [ ]:
# Now let's do model training as our data is ready
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# let's evaluate our model on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy Score on training Data: ", training_data_accuracy)

Accuracy Score on training Data:  0.79871953125


In [ ]:
# let's evaluate our model on testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("Accuracy Score on training Data: ", test_data_accuracy)

Accuracy Score on training Data:  0.77668125


In [ ]:
# model importing
file = "sentiment_model.pkl"
import pickle
pickle.dump(model, open(file, 'wb'))

vector_file = "vectorizer.pkl"
pickle.dump(vector, open(vector_file, 'wb'))

In [ ]:
# let's test our model by doing prediction
loaded_model = pickle.load(open('sentiment_model.pkl', 'rb'))

In [ ]:
X_new = X_test[0]
prediction = loaded_model.predict(X_new)
# print(prediction)
if (prediction[0] == 0):
  print("Oh No, It's A negative tweet")

else:
  print("Hurray, It's A positive tweet")

Hurray, It's A positive tweet
